## Combining the Fasttext + W2V (Fast 2 Vec)

In [ ]:
# mount your drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!gdown https://drive.google.com/uc?id=1BpTkrFqGH4BoDBTcR3tmm9ZZuj6FjlLB
!gdown https://drive.google.com/uc?id=1puCMovgqC32m9AAshbASzGHDEgKvWEvM
!gdown https://drive.google.com/uc?id=1NtebE9VUzA4MvA56EuBSN-OVl7CODHo1

Downloading...
From: https://drive.google.com/uc?id=1BpTkrFqGH4BoDBTcR3tmm9ZZuj6FjlLB
To: /content/requirements.txt
100% 273/273 [00:00<00:00, 948kB/s]
Downloading...
From: https://drive.google.com/uc?id=1puCMovgqC32m9AAshbASzGHDEgKvWEvM
To: /content/src.zip
100% 18.1k/18.1k [00:00<00:00, 35.1MB/s]
Downloading...
From (original): https://drive.google.com/uc?id=1NtebE9VUzA4MvA56EuBSN-OVl7CODHo1
From (redirected): https://drive.google.com/uc?id=1NtebE9VUzA4MvA56EuBSN-OVl7CODHo1&confirm=t&uuid=2ee94649-9cf8-4d5d-aa07-0a267b357391
To: /content/data.zip
100% 76.5M/76.5M [00:04<00:00, 17.0MB/s]


In [ ]:
# extract data and source code (src)
!unzip /content/src.zip -d /content/
!unzip /content/data.zip -d /content/

Archive:  /content/src.zip
 extracting: /content/src/__init__.py  
   creating: /content/src/__pycache__/
  inflating: /content/src/__pycache__/__init__.cpython-37.pyc  
  inflating: /content/src/__pycache__/static_method.cpython-37.pyc  
  inflating: /content/src/__pycache__/topic_detection.cpython-37.pyc  
  inflating: /content/src/__pycache__/topic_semantic.cpython-37.pyc  
  inflating: /content/src/static_method.py  
  inflating: /content/src/topic_detection.py  
  inflating: /content/src/topic_semantic.py  
Archive:  /content/data.zip
  inflating: /content/data/centroid_fasttext_menggunakan_stopwords.csv  
  inflating: /content/data/centroid_fasttext_w2v_menggunakan_stopwords.csv  
  inflating: /content/data/centroid_w2v_menggunakan_stopwords.csv  
  inflating: /content/data/data_jurnal_clean.csv  
  inflating: /content/data/data_jurnal_fix.csv  
  inflating: /content/data/hasil_cluster_fasttext_menggunakan_stopwords.csv  
  inflating: /content/data/hasil_cluster_w2v_menggunakan_s

In [ ]:
# create env and install package
!pip3 install virtualenv
!virtualenv /content/disertasi
!source /content/disertasi; pip install --upgrade pip
!source /content/disertasi; pip install ipykernel
!source /content/disertasi; pip install -r /content/requirements.txt

%cd /content/
%ls

# !pip install networkx
!pip install numpy==1.25.0

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 37.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.9/468.9 kB 30.5 MB/s eta 0:00:00
created virtual environment CPython3.10.12.final.0-64 in 2461ms
  creator CPython3Posix(dest=/content/disertasi, clear=False, no_vcs_ignore=False, global=False)
  seeder FromAppData(download=False, pip=bundle, setuptools=bundle, wheel=bundle, via=copy, app_data_dir=/root/.local/share/virtualenv)
    added seed packages: pip==24.2, setuptools==75.1.0, wheel==0.44.0
  activators BashActivator,CShellActivator,FishActivator,NushellActivator,PowerShellActivator,PythonActivator
/bin/bash: line 1: source: /content/disertasi: is a directory
  Using cached pip-24.2-py3-none-any.whl.metadata (3.6 kB)
Using cached pip-24.2-py3-none-any.whl (1.8 MB)
  Attempting uninstall: pip
    Found existing installation: pip 24.1.2
    Uninstalling pip-24.1.2:
      Successfully uninstalled pip-24.1.2
/bin/bash: line 1: source: /content/disertasi: is a d

## Stop here and restart your session

In [ ]:
import os
import pandas as pd
import numpy as np

from nltk.tokenize import word_tokenize
from gensim.models import Word2Vec, FastText
import gensim
from string import punctuation
import time

In [ ]:
import nltk
nltk.download('wordnet')
nltk.download('punkt')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

# Prepare Corpus

In [ ]:
df = pd.read_csv("/content/data/data_jurnal_clean.csv")
df.head()

,year,title,authors,abstract,abstract_clean
0,2023.0,PENERAPAN PROFILE MATCHING PADA SISTEM PENUNJA...,"Azrul Akbar, dan Safitri Juanita, Program Stud...",An essential resource in the company in achiev...,essential resource the company achieving goal ...
1,2023.0,Resource allocation optimization for mitigatin...,"Sheetal Bagali, Ramakrishnan Sundaraguru, Corr...",Wireless underwater sensor networks (UWSNs) ar...,wireless underwater sensor network uwsns are u...
2,2023.0,Virtual reality eye exercises application base...,"Komang Candra Brata, Hanifah Muslimah Az-Zahra...","In many situations, ophthalmologists prescribe...",many situation ophthalmologist prescribe glass...
3,2023.0,Transformation to a smart factory using NodeMC...,"Maryam Abdulhakeem Hailan, Baraa Munqith Albak...",Incorporating internet of things (IoT) in indu...,incorporating internet thing iot industrial sy...
4,2023.0,Two-level frame aggregation with enhanced A-MP...,"Fatoumata Sorra, Mohamed Othman, Umar Ali Buka...",One of the most important frame aggregation fe...,one the most important frame aggregation featu...


In [ ]:
df.shape

(54689, 5)

In [ ]:
stopwords = []
f = open('/content/data/stopwords.txt', 'r')
for i in f.readlines():
    stopwords.append(i.replace('\n', ''))
sw_eng = stopwords + list(punctuation)

In [ ]:
def extract_words(df, columns, stopwords=None):
    data = df.copy()
    if stopwords is not None:
        return [[word for word in word_tokenize(review.lower().strip()) if word not in stopwords] for review in data[columns]]
    return [word_tokenize(review.lower().strip()) for review in data[columns]]

In [ ]:
# list word with stopwords
words_with_sw = extract_words(df, 'abstract_clean', stopwords=sw_eng)

# list word without stopwords
words_without_sw = extract_words(df, 'abstract_clean', stopwords=None)

# Load

In [ ]:
ft_with_sw = FastText.load('/content/drive/MyDrive/Upload_Github/model/fasttext/model_fasttext_abstract_with_sw_win4.ft')
ft_without_sw = FastText.load('/content/drive/MyDrive/Upload_Github/model/fasttext/model_fasttext_abstract_without_sw_win4.ft')

w2v_with_sw = Word2Vec.load('/content/drive/MyDrive/Upload_Github/model/w2v/model_w2v_abstract_with_sw.w2v')
w2v_without_sw = Word2Vec.load('/content/drive/MyDrive/Upload_Github/model/w2v/model_w2v_abstract_without_sw.w2v')

The researcher chose the window size 4 setting on FastText. because it has the same window size settings as word 2 vector. The researcher chose a model using stop words to avoid including unnecessary words/noise

In [ ]:
ft_with_sw.wv.vectors

array([[ 6.2523484 , -4.9185295 ,  2.6807084 , ..., -0.4477684 ,
         0.5402266 ,  3.1499538 ],
       [-5.24207   , -0.28945756, -0.54483557, ...,  0.59262615,
         1.3937591 ,  6.0661483 ],
       [ 1.0157071 ,  0.3956688 ,  0.53017056, ...,  0.18790866,
        -0.49187624,  0.91273487],
       ...,
       [ 1.9890671 ,  4.472649  ,  1.6151751 , ...,  0.14974351,
         0.6176896 , -0.60048395],
       [ 2.051369  ,  2.3926566 , -0.4537618 , ...,  0.32407126,
         0.8652385 , -2.0146456 ],
       [-1.4743044 ,  1.1519783 ,  0.72020864, ...,  0.5411011 ,
         1.6262462 ,  2.1763127 ]], dtype=float32)

In [ ]:
ft_without_sw.wv.vectors

array([[-0.20845263, -1.9421272 ,  1.927397  , ...,  0.23304214,
         1.1322806 , -0.8713636 ],
       [ 3.5909047 , -0.32075104,  1.5026163 , ..., -2.3931775 ,
         2.69846   ,  2.333752  ],
       [-0.37029237,  0.32139453, -6.7241    , ..., -0.55720276,
         1.91461   , -2.3401911 ],
       ...,
       [-0.31442904, -0.8076426 ,  2.8259366 , ...,  0.2753022 ,
         1.4549272 , -0.241071  ],
       [-0.47552618,  2.6970406 , -0.98999226, ...,  1.4575459 ,
         0.01148456,  0.29971927],
       [ 0.95778877, -0.9713517 ,  1.5553291 , ..., -0.78995734,
         0.7581806 ,  0.577213  ]], dtype=float32)

In [ ]:
# modif with add matriks
# without stopwords
ft_without_sw.wv.vectors += w2v_without_sw.wv.vectors

# with stopwords
ft_with_sw.wv.vectors += w2v_with_sw.wv.vectors

In [ ]:
ft_with_sw.wv.vectors

array([[ 6.093137  , -3.9793112 ,  5.5478544 , ..., -2.0560791 ,
         1.5013912 ,  0.5087366 ],
       [-6.6609354 , -0.7206186 ,  1.0510288 , ...,  0.92942095,
         0.64400905,  6.201934  ],
       [ 1.8620609 ,  2.7099192 , -0.09200126, ...,  2.944469  ,
         1.8351299 , -1.8402427 ],
       ...,
       [ 3.3004262 ,  4.7095904 ,  2.563994  , ...,  0.09074511,
         0.712109  , -0.5466558 ],
       [ 1.6257669 ,  3.4363256 , -0.9140625 , ...,  1.1237642 ,
         1.1331421 , -3.5531402 ],
       [-2.6021993 ,  0.36682183,  0.93473506, ..., -0.80927   ,
         0.9420817 ,  1.7615402 ]], dtype=float32)

In [ ]:
ft_without_sw.wv.vectors

array([[-5.4180914e-01, -1.7782875e+00,  2.3966935e+00, ...,
        -1.7972702e+00,  2.1230483e+00, -9.3011856e-02],
       [ 4.0578432e+00, -1.5491539e+00,  1.3393599e+00, ...,
        -4.2937069e+00,  2.2790580e+00,  3.0980065e+00],
       [ 4.4771099e-01,  1.4871500e+00, -8.3276291e+00, ...,
        -8.1445599e-01,  4.1923442e+00, -3.6172504e+00],
       ...,
       [-5.0365382e-01, -7.4987847e-01,  2.7152164e+00, ...,
         1.5360326e-01,  1.9259752e+00,  7.0326328e-03],
       [-6.9538510e-01,  2.8310838e+00, -8.8566291e-01, ...,
         7.5401390e-01,  4.0136880e-01,  6.8058717e-01],
       [ 8.9468879e-01, -2.1492333e+00,  1.0131880e+00, ...,
        -3.6703563e-01,  6.1321974e-01,  6.0966933e-01]], dtype=float32)

In [ ]:
ft_with_sw.wv.vectors.shape, ft_without_sw.wv.vectors.shape

((32673, 100), (59694, 100))

# Model Information

In [ ]:
ft_with_sw.wv.key_to_index

{'student': 0,
 'system': 1,
 'analysis': 2,
 'model': 3,
 'value': 4,
 'test': 5,
 'level': 6,
 'performance': 7,
 'technique': 8,
 'effect': 9,
 'process': 10,
 'control': 11,
 'design': 12,
 'development': 13,
 'group': 14,
 'time': 15,
 'activity': 16,
 'problem': 17,
 'show': 18,
 'high': 19,
 'factor': 20,
 'out': 21,
 'power': 22,
 'approach': 23,
 'quality': 24,
 'significant': 25,
 'application': 26,
 'indonesia': 27,
 'information': 28,
 'number': 29,
 'obtained': 30,
 'algorithm': 31,
 'medium': 32,
 'education': 33,
 'sample': 34,
 'increase': 35,
 'determine': 36,
 'school': 37,
 'network': 38,
 'type': 39,
 'area': 40,
 'conducted': 41,
 'service': 42,
 'variable': 43,
 'teacher': 44,
 'work': 45,
 'different': 46,
 'need': 47,
 'material': 48,
 'into': 49,
 'purpose': 50,
 'good': 51,
 'but': 52,
 'community': 53,
 'management': 54,
 'implementation': 55,
 'form': 56,
 'social': 57,
 'well': 58,
 'condition': 59,
 'treatment': 60,
 'because': 61,
 'influence': 62,
 'peop

In [ ]:
ft_without_sw.wv.key_to_index

{'the': 0,
 'and': 1,
 'this': 2,
 'that': 3,
 'with': 4,
 'for': 5,
 'study': 6,
 'are': 7,
 'research': 8,
 'result': 9,
 'were': 10,
 'using': 11,
 'from': 12,
 'data': 13,
 'used': 14,
 'method': 15,
 'student': 16,
 'can': 17,
 'system': 18,
 'which': 19,
 'learning': 20,
 'analysis': 21,
 'based': 22,
 'model': 23,
 'have': 24,
 'not': 25,
 'value': 26,
 'one': 27,
 'there': 28,
 'test': 29,
 'level': 30,
 'between': 31,
 'performance': 32,
 'showed': 33,
 'technique': 34,
 'effect': 35,
 'been': 36,
 'process': 37,
 'control': 38,
 'also': 39,
 'through': 40,
 'paper': 41,
 'design': 42,
 'aim': 43,
 'development': 44,
 'group': 45,
 'time': 46,
 'activity': 47,
 'problem': 48,
 'show': 49,
 'their': 50,
 'use': 51,
 'high': 52,
 'factor': 53,
 'out': 54,
 'power': 55,
 'proposed': 56,
 'approach': 57,
 'quality': 58,
 'significant': 59,
 'application': 60,
 'indonesia': 61,
 'information': 62,
 'number': 63,
 'will': 64,
 'two': 65,
 'obtained': 66,
 'algorithm': 67,
 'such': 6

In [ ]:
ft_with_sw.wv['learning']

array([ 0.56832147, -1.5576314 ,  3.6235101 , -0.12144045,  0.7467816 ,
       -1.2555801 , -0.22758065, -2.183325  ,  1.8910155 , -2.9719622 ,
       -1.8068135 , -0.46529433, -0.90553445, -0.81324893, -0.57334167,
        0.03113797, -1.8877566 , -3.40834   ,  0.84119546, -0.2422252 ,
       -5.794825  ,  3.600072  ,  0.70466745,  0.3215255 , -0.6542982 ,
       -2.229712  , -2.9652426 ,  0.9560224 , -0.07478227,  0.16313879,
       -1.9801614 ,  1.8775071 ,  0.6505298 , -0.16393267,  0.9001763 ,
        1.1842697 , -0.6563259 ,  2.355823  , -0.80714357, -1.1166592 ,
       -0.3348446 ,  0.03674971, -1.9465555 , -2.8981776 , -0.27371457,
       -1.1584885 ,  3.9796896 , -1.1259801 , -1.4028509 , -0.70829296,
        0.400898  , -2.225105  ,  0.5144891 , -0.37188962,  1.4977812 ,
       -0.441104  , -2.738284  , -0.898504  , -4.7326746 , -2.2269397 ,
       -0.7965727 , -0.12537728, -1.5778394 ,  3.5036325 , -2.2275643 ,
       -0.78006864, -2.257593  ,  2.1856198 ,  1.6510468 ,  0.61

In [ ]:
ft_with_sw.wv['special']

array([ 1.7799163 ,  4.790617  ,  5.25345   , -2.6114502 ,  0.3603826 ,
        2.3034759 , -0.21332788,  0.42510927,  1.4209305 , -0.8146579 ,
       -2.0669565 ,  0.9401529 , -2.193891  ,  2.4990296 , -0.94016075,
       -3.142699  , -4.0115976 , -1.5520144 , -1.4692696 , -0.8320055 ,
        2.332895  , -0.16154149, -0.37926954,  1.6788177 ,  2.0221713 ,
       -5.178446  , -4.1794033 ,  0.6392567 ,  2.5733118 , -1.3202724 ,
       -5.5096197 , -1.5167817 ,  1.7594202 , -2.4973252 , -0.85292155,
        1.4282972 , -1.2561187 , -3.294665  ,  2.378453  ,  0.5413332 ,
       -3.195088  ,  0.6980045 , -2.5101855 ,  5.186021  ,  4.8816767 ,
        0.84069884,  0.11272097, -4.241253  ,  0.25412896,  3.0875258 ,
       -0.80420107,  6.1935873 , -1.468267  , -6.1575203 , -0.29571503,
        2.1061225 ,  3.8741934 ,  2.517537  ,  3.530629  , -3.0879004 ,
        1.515554  , -0.6847937 , -0.74072236,  1.0653799 , -0.774111  ,
       -0.8773527 , -1.7549316 ,  1.6924758 , -2.4373944 ,  0.32

# Sanity Check

## Similar by Word

## Without Stopwords

In [ ]:
ft_without_sw.wv.similar_by_word('study', topn=10)

[('research', 0.9173508882522583),
 ('researh', 0.8195320963859558),
 ('reseacrh', 0.8147439956665039),
 ('tudy', 0.8106340169906616),
 ('studyis', 0.8046499490737915),
 ('studywas', 0.7928076386451721),
 ('resesearch', 0.765135645866394),
 ('reseach', 0.7571027874946594),
 ('thestudy', 0.7527465224266052),
 ('researc', 0.7476922273635864)]

In [ ]:
ft_without_sw.wv.similar_by_word('learning', topn=10)

[('teaching', 0.8286192417144775),
 ('instructional', 0.812526285648346),
 ('learnig', 0.7950858473777771),
 ('lecture', 0.7649874091148376),
 ('elearning', 0.7635955214500427),
 ('learn', 0.7635181546211243),
 ('learnt', 0.7503476142883301),
 ('learningbased', 0.7467889189720154),
 ('thelearning', 0.7441290616989136),
 ('teachinglearning', 0.7358589172363281)]

In [ ]:
ft_without_sw.wv.similar_by_word('student', topn=10)

[('learner', 0.8438861966133118),
 ('teacher', 0.8362212181091309),
 ('studentsof', 0.8085640072822571),
 ('studentteachers', 0.7912940979003906),
 ('mathematics', 0.7691934108734131),
 ('ofstudent', 0.7602957487106323),
 ('astudents', 0.7600581049919128),
 ('thestudents', 0.7380731701850891),
 ('ofstudents', 0.735273003578186),
 ('studens', 0.7296404242515564)]

In [ ]:
ft_without_sw.wv.similar_by_word('network', topn=10)

[('networkswith', 0.7213286757469177),
 ('networking', 0.6940484046936035),
 ('networkbased', 0.680313766002655),
 ('networked', 0.67145174741745),
 ('subnetworks', 0.6429018378257751),
 ('node', 0.6259540915489197),
 ('wireless', 0.6153281927108765),
 ('system', 0.6071141958236694),
 ('computing', 0.5999178886413574),
 ('routing', 0.5997832417488098)]

In [ ]:
ft_without_sw.wv.similar_by_word('method', topn=10)

[('approach', 0.8479049801826477),
 ('technique', 0.8113372325897217),
 ('methodology', 0.7653294801712036),
 ('methodes', 0.716752827167511),
 ('methodelogy', 0.7130810618400574),
 ('methode', 0.6917364597320557),
 ('mixedmethod', 0.6876387596130371),
 ('methodswith', 0.678158164024353),
 ('methodwas', 0.6733463406562805),
 ('aproach', 0.6617223024368286)]

In [ ]:
ft_without_sw.wv.similar_by_word('paper', topn=10)

[('paperwork', 0.7431836128234863),
 ('pape', 0.7291826009750366),
 ('article', 0.7115405797958374),
 ('paperuses', 0.7064417004585266),
 ('paperless', 0.6630925536155701),
 ('paperbased', 0.6626742482185364),
 ('flypaper', 0.6208834648132324),
 ('study', 0.6060547232627869),
 ('wallpaper', 0.5799899697303772),
 ('sandpaper', 0.5728821158409119)]

In [ ]:
ft_without_sw.wv.similar_by_word('research', topn=10)

[('study', 0.9173508882522583),
 ('researh', 0.7826224565505981),
 ('reseach', 0.7765402793884277),
 ('reseacrh', 0.7350415587425232),
 ('researchis', 0.7109057307243347),
 ('reseaarch', 0.7101735472679138),
 ('researchwas', 0.7017461061477661),
 ('esearch', 0.6993891596794128),
 ('studyis', 0.69712895154953),
 ('thisresearch', 0.6908573508262634)]

In [ ]:
ft_without_sw.wv.similar_by_word('computer', topn=10)

[('minicomputer', 0.6813381314277649),
 ('microcomputer', 0.6518404483795166),
 ('plc', 0.6258369088172913),
 ('raspberry', 0.6111272573471069),
 ('software', 0.6078949570655823),
 ('hardware', 0.6008398532867432),
 ('virtual', 0.5996378660202026),
 ('desktop', 0.5971086621284485),
 ('advanced', 0.5946280360221863),
 ('embedded', 0.5860965251922607)]

In [ ]:
ft_without_sw.wv.similar_by_word('medical', topn=10)

[('medic', 0.6951238512992859),
 ('biomedical', 0.6732518076896667),
 ('healthcare', 0.6529082655906677),
 ('medicare', 0.6405441164970398),
 ('nursing', 0.6232755184173584),
 ('clinical', 0.6150152087211609),
 ('medically', 0.6088822484016418),
 ('medication', 0.6037389039993286),
 ('oncology', 0.5997435450553894),
 ('swamedication', 0.5850264430046082)]

In [ ]:
ft_without_sw.wv.similar_by_word('special', topn=10)

[('especial', 0.6074124574661255),
 ('particular', 0.6027982831001282),
 ('esspecially', 0.551725447177887),
 ('general', 0.5505132079124451),
 ('inclusive', 0.5393542051315308),
 ('specialy', 0.5355970859527588),
 ('enteral', 0.5285974144935608),
 ('specialized', 0.5262109041213989),
 ('communicable', 0.5257035493850708),
 ('adequate', 0.5238739848136902)]

### With Stopwords

#### in of vocabulary (match the word 2 vector results)

In [ ]:
ft_with_sw.wv.similar_by_word('student', topn=10)

[('learner', 0.8217413425445557),
 ('teacher', 0.8141673803329468),
 ('mathematics', 0.7775900363922119),
 ('achievement', 0.6823602914810181),
 ('lecturer', 0.6817526817321777),
 ('teaching', 0.6736299991607666),
 ('participant', 0.6701919436454773),
 ('classroom', 0.6667459011077881),
 ('skill', 0.6650975942611694),
 ('educator', 0.6277675032615662)]

In [ ]:
ft_with_sw.wv.similar_by_word('network', topn=10)

[('networking', 0.6821368336677551),
 ('networked', 0.6800596714019775),
 ('server', 0.6569830179214478),
 ('link', 0.635098934173584),
 ('wireless', 0.6265502572059631),
 ('channel', 0.6166669130325317),
 ('routing', 0.6156995296478271),
 ('node', 0.6112644076347351),
 ('connection', 0.6053950190544128),
 ('system', 0.6032312512397766)]

In [ ]:
ft_with_sw.wv.similar_by_word('computer', topn=10)

[('microcomputer', 0.5933501124382019),
 ('software', 0.577326774597168),
 ('mechatronic', 0.5704676508903503),
 ('gpu', 0.560751736164093),
 ('smartphone', 0.5592344403266907),
 ('desktop', 0.5516309142112732),
 ('multisensor', 0.5469931960105896),
 ('komputer', 0.5409208536148071),
 ('virtualized', 0.540777862071991),
 ('interface', 0.5396437644958496)]

In [ ]:
ft_with_sw.wv.similar_by_word('medical', topn=10)

[('medically', 0.656035840511322),
 ('healthcare', 0.6465635895729065),
 ('biomedical', 0.6422223448753357),
 ('medic', 0.632082998752594),
 ('nursing', 0.6013479232788086),
 ('telemedicine', 0.5758368372917175),
 ('surgical', 0.5722725987434387),
 ('physician', 0.5620725154876709),
 ('diagnosis', 0.5522878170013428),
 ('selfmedication', 0.5514518022537231)]

In [ ]:
ft_with_sw.wv.similar_by_word('school', topn=10)

[('labschool', 0.7629711031913757),
 ('junior', 0.7057497501373291),
 ('preschool', 0.6757100224494934),
 ('senior', 0.6707327365875244),
 ('homeschool', 0.665015697479248),
 ('teacher', 0.6594382524490356),
 ('education', 0.6550800204277039),
 ('yunior', 0.625357985496521),
 ('scholl', 0.6095897555351257),
 ('schooling', 0.6050417423248291)]

In [ ]:
ft_with_sw.wv.similar_by_word('architecture', topn=10)

[('framework', 0.6965734958648682),
 ('architect', 0.6227698922157288),
 ('computing', 0.6046209931373596),
 ('autoencoder', 0.5971618890762329),
 ('hardware', 0.5966168642044067),
 ('scheme', 0.5928981304168701),
 ('system', 0.5920132398605347),
 ('platform', 0.589621365070343),
 ('topology', 0.5865606665611267),
 ('modular', 0.5799277424812317)]

In [ ]:
ft_with_sw.wv.similar_by_word('artificial', topn=10)

[('artificially', 0.5561100244522095),
 ('swarm', 0.5222353339195251),
 ('bat', 0.5095996856689453),
 ('metaheuristic', 0.47198957204818726),
 ('intelligent', 0.44900229573249817),
 ('evolutionary', 0.4239146113395691),
 ('optimizes', 0.4224812090396881),
 ('troop', 0.4136352837085724),
 ('hybridization', 0.4103996455669403),
 ('ffnn', 0.40871578454971313)]

In [ ]:
ft_with_sw.wv.similar_by_word('science', topn=10)

[('scientific', 0.6866530179977417),
 ('sociology', 0.6680729389190674),
 ('physic', 0.6498183608055115),
 ('teaching', 0.6460927724838257),
 ('mathematics', 0.6279463171958923),
 ('conscience', 0.6217556595802307),
 ('literacy', 0.6201630234718323),
 ('ethnoscience', 0.6150078773498535),
 ('biology', 0.5904272198677063),
 ('chemistry', 0.5899773836135864)]

In [ ]:
ft_with_sw.wv.similar_by_word('engineering', topn=10)

[('mechatronics', 0.6052939295768738),
 ('engineer', 0.6031684279441833),
 ('mechatronic', 0.5893663167953491),
 ('automotive', 0.5521070957183838),
 ('electronics', 0.5430510640144348),
 ('mechanical', 0.5391537547111511),
 ('industrial', 0.5139726996421814),
 ('computer', 0.5105270147323608),
 ('reengineering', 0.5077874064445496),
 ('mechanic', 0.5063520073890686)]

In [ ]:
ft_with_sw.wv.similar_by_word('academic', topn=10)

[('nonacademic', 0.738606870174408),
 ('lecturer', 0.6473168134689331),
 ('undergraduate', 0.6440005302429199),
 ('achievement', 0.6298796534538269),
 ('thesis', 0.6187666058540344),
 ('student', 0.5937726497650146),
 ('college', 0.584913432598114),
 ('academically', 0.5846592783927917),
 ('motivation', 0.5829130411148071),
 ('school', 0.5806761980056763)]

#### OOV (Out of Vocabulary)

In [ ]:
ft_with_sw.wv.similar_by_word('learn', topn=10)

[('learnig', 0.6827137470245361),
 ('learnt', 0.5955535769462585),
 ('teachinglearning', 0.5437812805175781),
 ('learned', 0.528630793094635),
 ('learing', 0.5119184255599976),
 ('thelearning', 0.4959978759288788),
 ('learningbased', 0.48937493562698364),
 ('instructing', 0.48883429169654846),
 ('writhing', 0.4823170602321625),
 ('enquiry', 0.4646671414375305)]

In [ ]:
ft_with_sw.wv.similar_by_word('for', topn=10)

[('fort', 0.5429442524909973),
 ('fore', 0.5418683290481567),
 ('tor', 0.5392280220985413),
 ('forever', 0.5227044820785522),
 ('foreigner', 0.5194446444511414),
 ('resignation', 0.51072758436203),
 ('forerunner', 0.5052616000175476),
 ('propositional', 0.5026342272758484),
 ('liberalization', 0.4987903833389282),
 ('theorist', 0.4888688623905182)]

In [ ]:
ft_with_sw.wv.similar_by_word('research', topn=10)

[('esearch', 0.8246558904647827),
 ('thisresearch', 0.690599262714386),
 ('researching', 0.6670873165130615),
 ('onesearch', 0.6621724963188171),
 ('researched', 0.5670267939567566),
 ('literatur', 0.5445489287376404),
 ('searching', 0.5326306223869324),
 ('researcher', 0.5110146403312683),
 ('cyberliterature', 0.4756118953227997),
 ('search', 0.47407254576683044)]

In [ ]:
ft_with_sw.wv.similar_by_word('most', topn=10)

[('since', 0.45507165789604187),
 ('ost', 0.454382985830307),
 ('welch', 0.42405804991722107),
 ('heartwood', 0.4222324788570404),
 ('remarkably', 0.40069639682769775),
 ('majorly', 0.39757630228996277),
 ('burnt', 0.3928661644458771),
 ('dipterocarpaceae', 0.39154285192489624),
 ('hst', 0.3850105106830597),
 ('sdr', 0.3841559588909149)]

can be seen in the results above. that fasttext modification is not only based on subwords. but can capture semantics even in words outside the vocab / Out of vocabulary.

It can be concluded that by combining fasttext with word2vector, fasttext can better understand semantic context. However, still with fasttext capabilities that can handle OOV (Out of Vocabulary).

## Save Model Kombinasi

In [ ]:
from jcopml.utils import save_model, load_model

In [ ]:
save_model(ft_with_sw.wv, 'model_fasttext_abstract_with_sw_modif.pkl', folder_name='/content/drive/MyDrive/Upload_Github/model/fasttext_modif/')
save_model(ft_without_sw.wv, 'model_fasttext_abstract_without_sw_modif.pkl', folder_name='/content/drive/MyDrive/Upload_Github/model/fasttext_modif/')

Model is pickled as /content/drive/MyDrive/Upload_Github/model/fasttext_modif//model_fasttext_abstract_with_sw_modif.pkl
Model is pickled as /content/drive/MyDrive/Upload_Github/model/fasttext_modif//model_fasttext_abstract_without_sw_modif.pkl


In [ ]:
ft_with_sw = load_model('/content/drive/MyDrive/Upload_Github/model/fasttext_modif/model_fasttext_abstract_with_sw_modif.pkl')
ft_without_sw = load_model('/content/drive/MyDrive/Upload_Github/model/fasttext_modif/model_fasttext_abstract_without_sw_modif.pkl')

## Higher Order Visualization

In [ ]:
from umap import UMAP
from sklearn.manifold import TSNE
import plotly.express as px

In [ ]:
X_with_sw = UMAP().fit_transform(ft_with_sw.vectors)
X_without_sw = UMAP().fit_transform(ft_without_sw.vectors)

In [ ]:
df_with_sw = pd.DataFrame(X_with_sw, columns=['umap_1','umap_2'])
df_with_sw['word'] = ft_with_sw.index_to_key

In [ ]:
df_without_sw = pd.DataFrame(X_without_sw, columns=['umap_1','umap_2'])
df_without_sw['word'] = ft_without_sw.index_to_key

## Visualization Fast 2 Vec with Stopwords

In [ ]:
fig = px.scatter(df_with_sw, text='word', x='umap_1', y='umap_2')
fig.update_traces(textposition='top center')
fig.update_layout(
    height=800,
    title_text='Reduced Visualization with Stopwords'
)
fig.show()

## Visualization Fast 2 Vec WithOut Stopwords

In [ ]:
fig = px.scatter(df_without_sw, text='word', x='umap_1', y='umap_2')
fig.update_traces(textposition='top center')
fig.update_layout(
    height=800,
    title_text='Reduced Visualization without Stopwords'
)
fig.show()

Output hidden; open in https://colab.research.google.com to view.

### Kalimat 1
Mekanisme pengujian adalah membandingkan kemiripan vektor antara susunan kata hasil dari manusia (label) dengan model embedding (pred).

In [ ]:
from scipy import spatial

In [ ]:
# load model embedding
ft_with_sw1 = ft_with_sw

In [ ]:
# transform kalimat to vektor
def norm_sent_vector(sentences, wv_model):
    vecs = [wv_model[word.lower()] for word in word_tokenize(sentences)]
    norm_vecs = [vec / np.linalg.norm(vec) for vec in vecs if np.linalg.norm(vec) > 0]
    sent_vect = np.mean(norm_vecs, axis=0)
    return sent_vect

In [ ]:
# persiapkan label dan prediksi
pred = [i[0] for i in ft_with_sw1.similar_by_word('student', topn=10)]
pred = ' '.join(pred)

In [ ]:
label = 'teacher, learner, mathematics, achievement, participant, classroom, skill, teaching, lecturer, educator'
label = label.replace(',', '')

In [ ]:
pred

'learner teacher mathematics achievement lecturer teaching participant classroom skill educator'

In [ ]:
label

'teacher learner mathematics achievement participant classroom skill teaching lecturer educator'

In [ ]:
vectors_pred = norm_sent_vector(pred, ft_with_sw1)
vectors_label = norm_sent_vector(label, ft_with_sw1)

In [ ]:
result_1 = 1 - spatial.distance.cosine(vectors_pred, vectors_label)
result_1

0.9999998807907104

### Kalimat 2

In [ ]:
# persiapkan label dan prediksi
pred = [i[0] for i in ft_with_sw1.similar_by_word('network', topn=10)]
pred = ' '.join(pred)

In [ ]:
label = 'connection, system, node, server, routing, wireless, interconnection, architecture, device, networking'
label = label.replace(',', '')

In [ ]:
pred

'networking networked server link wireless channel routing node connection system'

In [ ]:
label

'connection system node server routing wireless interconnection architecture device networking'

In [ ]:
vectors_pred = norm_sent_vector(pred, ft_with_sw1)
vectors_label = norm_sent_vector(label, ft_with_sw1)

In [ ]:
result_2 = 1 - spatial.distance.cosine(vectors_pred, vectors_label)
result_2

0.9678806662559509

### Kalimat 3

In [ ]:
# persiapkan label dan prediksi
pred = [i[0] for i in ft_with_sw1.similar_by_word('computer', topn=10)]
pred = ' '.join(pred)

In [ ]:
label = 'software, desktop, gpu, interface, plc, automation, engineering, virtualized, digital, internet'
label = label.replace(',', '')

In [ ]:
pred

'microcomputer software mechatronic gpu smartphone desktop multisensor komputer virtualized interface'

In [ ]:
label

'software desktop gpu interface plc automation engineering virtualized digital internet'

In [ ]:
vectors_pred = norm_sent_vector(pred, ft_with_sw1)
vectors_label = norm_sent_vector(label, ft_with_sw1)

In [ ]:
result_3 = 1 - spatial.distance.cosine(vectors_pred, vectors_label)
result_3

0.9305719137191772

### Kalimat 4

In [ ]:
# persiapkan label dan prediksi
pred = [i[0] for i in ft_with_sw1.similar_by_word('medical', topn=10)]
pred = ' '.join(pred)

In [ ]:
label = 'healthcare, biomedical, diagnosis, doctor, nursing, specialist, surgical, cardiology, recorder, telemedicine'
label = label.replace(',', '')

In [ ]:
pred

'medically healthcare biomedical medic nursing telemedicine surgical physician diagnosis selfmedication'

In [ ]:
label

'healthcare biomedical diagnosis doctor nursing specialist surgical cardiology recorder telemedicine'

In [ ]:
vectors_pred = norm_sent_vector(pred, ft_with_sw1)
vectors_label = norm_sent_vector(label, ft_with_sw1)

In [ ]:
result_4 = 1 - spatial.distance.cosine(vectors_pred, vectors_label)
result_4

0.9407744407653809

### Kalimat 5

In [ ]:
# persiapkan label dan prediksi
pred = [i[0] for i in ft_with_sw1.similar_by_word('school', topn=10)]
pred = ' '.join(pred)

In [ ]:
label = 'teacher, junior, senior, education, academic, college, student, labschool, homeschool, smkn'
label = label.replace(',', '')

In [ ]:
pred

'labschool junior preschool senior homeschool teacher education yunior scholl schooling'

In [ ]:
label

'teacher junior senior education academic college student labschool homeschool smkn'

In [ ]:
vectors_pred = norm_sent_vector(pred, ft_with_sw1)
vectors_label = norm_sent_vector(label, ft_with_sw1)

In [ ]:
result_5 = 1 - spatial.distance.cosine(vectors_pred, vectors_label)
result_5

0.9522593021392822

### Kalimat 6

In [ ]:
# persiapkan label dan prediksi
pred = [i[0] for i in ft_with_sw1.similar_by_word('architecture', topn=10)]
pred = ' '.join(pred)

In [ ]:
label = 'framework, computing, system, topology, hardware, platform, computing, autoencoder, convolutional, scalable'
label = label.replace(',', '')

In [ ]:
pred

'framework architect computing autoencoder hardware scheme system platform topology modular'

In [ ]:
label

'framework computing system topology hardware platform computing autoencoder convolutional scalable'

In [ ]:
vectors_pred = norm_sent_vector(pred, ft_with_sw1)
vectors_label = norm_sent_vector(label, ft_with_sw1)

In [ ]:
result_6 = 1 - spatial.distance.cosine(vectors_pred, vectors_label)
result_6

0.9650278687477112

### Kalimat 7

In [ ]:
# persiapkan label dan prediksi
pred = [i[0] for i in ft_with_sw1.similar_by_word('artificial', topn=10)]
pred = ' '.join(pred)

In [ ]:
label = 'intelligent, swarm, metaheuristic, evolutionary, neurofuzzy, hybridization, ffnn, optimizer, troop, machine'
label = label.replace(',', '')

In [ ]:
pred

'artificially swarm bat metaheuristic intelligent evolutionary optimizes troop hybridization ffnn'

In [ ]:
label

'intelligent swarm metaheuristic evolutionary neurofuzzy hybridization ffnn optimizer troop machine'

In [ ]:
vectors_pred = norm_sent_vector(pred, ft_with_sw1)
vectors_label = norm_sent_vector(label, ft_with_sw1)

In [ ]:
result_7 = 1 - spatial.distance.cosine(vectors_pred, vectors_label)
result_7

0.9483245611190796

### Kalimat 8

In [ ]:
# persiapkan label dan prediksi
pred = [i[0] for i in ft_with_sw1.similar_by_word('science', topn=10)]
pred = ' '.join(pred)

In [ ]:
label = 'mathematics, biology, physic, chemistry, literacy, ethnoscience, geography, sociology, ethnoscience, inquiry'
label = label.replace(',', '')

In [ ]:
pred

'scientific sociology physic teaching mathematics conscience literacy ethnoscience biology chemistry'

In [ ]:
label

'mathematics biology physic chemistry literacy ethnoscience geography sociology ethnoscience inquiry'

In [ ]:
vectors_pred = norm_sent_vector(pred, ft_with_sw1)
vectors_label = norm_sent_vector(label, ft_with_sw1)

In [ ]:
result_8 = 1 - spatial.distance.cosine(vectors_pred, vectors_label)
result_8

0.9651998281478882

### Kalimat 9

In [ ]:
# persiapkan label dan prediksi
pred = [i[0] for i in ft_with_sw1.similar_by_word('engineering', topn=10)]
pred = ' '.join(pred)

In [ ]:
label = 'mechatronic, automotive, electronics, computer, industrial, informatics, robotics, mechanical, science, mechanic'
label = label.replace(',', '')

In [ ]:
pred

'mechatronics engineer mechatronic automotive electronics mechanical industrial computer reengineering mechanic'

In [ ]:
label

'mechatronic automotive electronics computer industrial informatics robotics mechanical science mechanic'

In [ ]:
vectors_pred = norm_sent_vector(pred, ft_with_sw1)
vectors_label = norm_sent_vector(label, ft_with_sw1)

In [ ]:
result_9 = 1 - spatial.distance.cosine(vectors_pred, vectors_label)
result_9

0.9480907917022705

### Kalimat 10

In [ ]:
# persiapkan label dan prediksi
pred = [i[0] for i in ft_with_sw1.similar_by_word('academic', topn=10)]
pred = ' '.join(pred)

In [ ]:
label = 'lecturer, student, accordance, achievement, career, thesis, school, writing, motivation, undergraduate'
label = label.replace(',', '')

In [ ]:
pred

'nonacademic lecturer undergraduate achievement thesis student college academically motivation school'

In [ ]:
label

'lecturer student accordance achievement career thesis school writing motivation undergraduate'

In [ ]:
vectors_pred = norm_sent_vector(pred, ft_with_sw1)
vectors_label = norm_sent_vector(label, ft_with_sw1)

In [ ]:
result_10 = 1 - spatial.distance.cosine(vectors_pred, vectors_label)
result_10

0.9563308358192444

### Kalimat 11

In [ ]:
# persiapkan label dan prediksi
pred = [i[0] for i in ft_with_sw1.similar_by_word('learn', topn=10)]
pred = ' '.join(pred)

In [ ]:
label = 'teaching, instructing, enquiry, writhing, teachinglearning, learningbased, learned, teach, thelearning, learnt'
label = label.replace(',', '')

In [ ]:
pred

'learnig learnt teachinglearning learned learing thelearning learningbased instructing writhing enquiry'

In [ ]:
label

'teaching instructing enquiry writhing teachinglearning learningbased learned teach thelearning learnt'

In [ ]:
vectors_pred = norm_sent_vector(pred, ft_with_sw1)
vectors_label = norm_sent_vector(label, ft_with_sw1)

In [ ]:
result_11 = 1 - spatial.distance.cosine(vectors_pred, vectors_label)
result_11

0.9731163382530212

### Kalimat 12

In [ ]:
# persiapkan label dan prediksi
pred = [i[0] for i in ft_with_sw1.similar_by_word('for', topn=10)]
pred = ' '.join(pred)

In [ ]:
label = 'resignation, propositional, liberalization, theorist, forerunner, foreigner, forever, fort, forwarder, forth'
label = label.replace(',', '')

In [ ]:
pred

'fort fore tor forever foreigner resignation forerunner propositional liberalization theorist'

In [ ]:
label

'resignation propositional liberalization theorist forerunner foreigner forever fort forwarder forth'

In [ ]:
vectors_pred = norm_sent_vector(pred, ft_with_sw1)
vectors_label = norm_sent_vector(label, ft_with_sw1)

In [ ]:
result_12 = 1 - spatial.distance.cosine(vectors_pred, vectors_label)
result_12

0.9608849287033081

### Kalimat 13

In [ ]:
# persiapkan label dan prediksi
pred = [i[0] for i in ft_with_sw1.similar_by_word('research', topn=10)]
pred = ' '.join(pred)

In [ ]:
label = 'literatur, esearch, thisresearch, researching, onesearch, researcher, researched, cyberliterature, search, searching'
label = label.replace(',', '')

In [ ]:
pred

'esearch thisresearch researching onesearch researched literatur searching researcher cyberliterature search'

In [ ]:
label

'literatur esearch thisresearch researching onesearch researcher researched cyberliterature search searching'

In [ ]:
vectors_pred = norm_sent_vector(pred, ft_with_sw1)
vectors_label = norm_sent_vector(label, ft_with_sw1)

In [ ]:
result_13 = 1 - spatial.distance.cosine(vectors_pred, vectors_label)
result_13

1

### Kalimat 14

In [ ]:
# persiapkan label dan prediksi
pred = [i[0] for i in ft_with_sw1.similar_by_word('most', topn=10)]
pred = ' '.join(pred)

In [ ]:
label = 'since, longest, frequently, half, mount, ghost, burnt, majorly, heartwood, remarkably'
label = label.replace(',', '')

In [ ]:
pred

'since ost welch heartwood remarkably majorly burnt dipterocarpaceae hst sdr'

In [ ]:
label

'since longest frequently half mount ghost burnt majorly heartwood remarkably'

In [ ]:
vectors_pred = norm_sent_vector(pred, ft_with_sw1)
vectors_label = norm_sent_vector(label, ft_with_sw1)

In [ ]:
result_14 = 1 - spatial.distance.cosine(vectors_pred, vectors_label)
result_14

0.8496797680854797

### Hitung Skor Rata-Rata Pengujian

In [ ]:
final_results = np.mean([result_1, result_2, result_3, result_4, result_5, result_6, result_7, result_8, result_9, result_10, result_11, result_12, result_13, result_14])
final_results

0.9541529374463218